In [45]:
import scipy
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import jaccard_score
from sklearn.metrics import log_loss

In [46]:
%pip install --user pandas
import pandas as pd
import numpy as np

df = pd.read_excel(r"C:\Users\USUARIO\Desktop\quiniela origen.xlsx")

Note: you may need to restart the kernel to use updated packages.


In [47]:
df.drop(['Temporada','Local', 'Visitante', 'V','V.1','E','E.1','D','D.1', 'sin perder', 'sin ganar', 'sin perder.1', 'sin ganar.1'], axis=1, inplace=True)

In [48]:
df= df.dropna(subset=['resultado'], axis=0)

In [49]:
df.drop(['Jornada', 'dif.', 'dif..1'], axis=1, inplace=True)

In [50]:
df['Puntos Local']=pd.cut(df['PUNTOS'], bins=3, labels=['menos de 3', 'de 3 a 5', '6 o más'], right=False)
df['Puntos Visitante']=pd.cut(df['PUNTOS.1'], bins=3, labels=['menos de 3', 'de 3 a 5', '6 o más'], right=False)

In [51]:
df['MPSP_l']=pd.qcut(df['partidos consecutivos sin perder'],q= [0, 0.4, 1])
df['MPSG_l']=pd.qcut(df['media de partidos sin ganar'], q= [0, 0.4, 1])
df['MPSP_v']=pd.qcut(df['partidos consecutivos sin perder.1'], q= [0, 0.4, 1])
df['MPSG_v']=pd.qcut(df['media de partidos sin ganar.1'], q= [0, 0.4, 1])

In [52]:
df['GF_local'] = pd.qcut(df['AF'], q= [0, 0.6, 1], labels = ['0-1.33', '>1.33'])
df['GC_local'] = pd.qcut(df['AC'], q= [0, 0.7, 1])

In [53]:
df['GF_visitante'] = pd.qcut(df['AF.1'], q= [0, 0.6, 1], labels = ['0-1.1', '>1.1'])
df['GC_visitante'] = pd.qcut(df['AC.1'], q= [0, 0.55, 1], labels = ['0-1.1', '>1.4'])
df

,cuartil,partidos consecutivos sin perder,media de partidos sin ganar,PUNTOS,último partido como local,último partido,AF,AC,cuartil.1,partidos consecutivos sin perder.1,...,Puntos Local,Puntos Visitante,MPSP_l,MPSG_l,MPSP_v,MPSG_v,GF_local,GC_local,GF_visitante,GC_visitante
0,1,6.800000,1.333333,9,3,3,2.380000,0.250000,2,4.142857,...,6 o más,6 o más,"(2.27, 12.0]","(-0.001, 2.25]","(2.233, 12.0]","(-0.001, 2.25]",>1.33,"(-0.001, 1.309]",>1.1,0-1.1
1,1,6.800000,1.300000,6,0,3,1.480000,1.190000,2,1.666667,...,6 o más,menos de 3,"(2.27, 12.0]","(-0.001, 2.25]","(0.999, 2.233]","(2.25, 11.0]",0-1.33,"(-0.001, 1.309]",0-1.1,>1.4
2,1,4.428571,1.555556,7,1,1,2.420000,1.080000,4,2.000000,...,6 o más,de 3 a 5,"(2.27, 12.0]","(-0.001, 2.25]","(0.999, 2.233]","(2.25, 11.0]",>1.33,"(-0.001, 1.309]",0-1.1,0-1.1
3,3,2.400000,2.250000,4,3,3,1.380000,0.630000,1,4.428571,...,de 3 a 5,6 o más,"(2.27, 12.0]","(-0.001, 2.25]","(2.233, 12.0]","(-0.001, 2.25]",0-1.33,"(-0.001, 1.309]",>1.1,0-1.1
4,3,2.555556,2.900000,4,0,0,1.640000,1.560000,1,6.800000,...,de 3 a 5,6 o más,"(2.27, 12.0]","(2.25, 11.0]","(2.233, 12.0]","(-0.001, 2.25]",>1.33,"(1.309, 4.0]",>1.1,0-1.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,3,2.000000,2.000000,4,3,1,1.333333,1.000000,4,1.500000,...,de 3 a 5,6 o más,"(0.999, 2.27]","(-0.001, 2.25]","(0.999, 2.233]","(-0.001, 2.25]",0-1.33,"(-0.001, 1.309]",0-1.1,>1.4
778,1,12.000000,0.000000,9,3,3,2.600000,1.000000,4,1.500000,...,6 o más,menos de 3,"(2.27, 12.0]","(-0.001, 2.25]","(0.999, 2.233]","(2.25, 11.0]",>1.33,"(-0.001, 1.309]",0-1.1,>1.4
779,2,2.250000,2.000000,1,0,0,1.333333,0.666667,2,2.000000,...,menos de 3,6 o más,"(0.999, 2.27]","(-0.001, 2.25]","(0.999, 2.233]","(-0.001, 2.25]",0-1.33,"(-0.001, 1.309]",0-1.1,0-1.1
780,2,2.666667,1.666667,6,3,0,1.200000,1.200000,2,2.250000,...,6 o más,de 3 a 5,"(2.27, 12.0]","(-0.001, 2.25]","(2.233, 12.0]","(-0.001, 2.25]",0-1.33,"(-0.001, 1.309]",0-1.1,0-1.1


# Histórico

## Dos años anteriores

---

In [54]:
chisq_cuartil = pd.crosstab(df['cuartil'], df['resultado'])
chisq_cuartil

resultado,0,1,2
cuartil,,,
1,31,89,20
2,45,81,49
3,80,90,81
4,62,81,73


In [55]:
scipy.stats.contingency.chi2_contingency(chisq_cuartil)

(35.13565011109789,
 4.056397041509435e-06,
 6,
 array([[ 39.02813299,  61.04859335,  39.92327366],
        [ 48.78516624,  76.31074169,  49.90409207],
        [ 69.97186701, 109.45140665,  71.57672634],
        [ 60.21483376,  94.18925831,  61.59590793]]))

---

In [56]:
chisq_cuartil_1 = pd.crosstab(df['cuartil.1'], df['resultado'])
chisq_cuartil_1

resultado,0,1,2
cuartil.1,,,
1,34,26,78
2,46,80,48
3,85,106,62
4,53,129,35


In [57]:
scipy.stats.contingency.chi2_contingency(chisq_cuartil_1)

(87.94830063626274,
 8.079412259415815e-17,
 6,
 array([[ 38.47058824,  60.17647059,  39.35294118],
        [ 48.50639386,  75.87468031,  49.61892583],
        [ 70.52941176, 110.32352941,  72.14705882],
        [ 60.49360614,  94.62531969,  61.88107417]]))

---

# Racha

## Tres últimos partidos
---

In [58]:
chisq_PUNTOS_L = pd.crosstab(df['PUNTOS'], df['resultado'])
chisq_PUNTOS_L

resultado,0,1,2
PUNTOS,,,
0,18,15,13
1,29,36,31
2,17,21,29
3,39,58,33
4,42,81,51
5,16,22,16
6,26,44,23
7,18,30,18
9,13,34,9


In [59]:
scipy.stats.contingency.chi2_contingency(chisq_PUNTOS_L)

(22.66219184488437,
 0.12306762743197185,
 16,
 array([[12.82352941, 20.05882353, 13.11764706],
        [26.76214834, 41.86189258, 27.37595908],
        [18.67774936, 29.21611253, 19.10613811],
        [36.24040921, 56.68797954, 37.07161125],
        [48.50639386, 75.87468031, 49.61892583],
        [15.05370844, 23.54731458, 15.39897698],
        [25.9258312 , 40.55370844, 26.52046036],
        [18.39897698, 28.78005115, 18.82097187],
        [15.6112532 , 24.41943734, 15.96930946]]))

---

In [60]:
chisq_PUNTOS_Lo = pd.crosstab(df['Puntos Local'], df['resultado'])
chisq_PUNTOS_Lo

resultado,0,1,2
Puntos Local,,,
menos de 3,64,72,73
de 3 a 5,97,161,100
6 o más,57,108,50


In [61]:
scipy.stats.contingency.chi2_contingency(chisq_PUNTOS_Lo)

(12.265973639597007,
 0.015479220548034052,
 4,
 array([[ 58.26342711,  91.13682864,  59.59974425],
        [ 99.80051151, 156.10997442, 102.08951407],
        [ 59.93606138,  93.75319693,  61.31074169]]))

---

In [62]:
chisq_PUNTOS_V = pd.crosstab(df['PUNTOS.1'], df['resultado'])
chisq_PUNTOS_V

resultado,0,1,2
PUNTOS.1,,,
0,6,22,7
1,31,45,20
2,23,35,13
3,41,51,28
4,39,63,42
5,15,25,17
6,28,46,35
7,29,43,37
9,6,11,24


In [63]:
scipy.stats.contingency.chi2_contingency(chisq_PUNTOS_V)
#p-value= 0.0002
#critical value = 29.7

(35.38667219249776,
 0.0035172195393826586,
 16,
 array([[ 9.75703325, 15.26214834,  9.98081841],
        [26.76214834, 41.86189258, 27.37595908],
        [19.79283887, 30.96035806, 20.24680307],
        [33.45268542, 52.32736573, 34.21994885],
        [40.14322251, 62.79283887, 41.06393862],
        [15.89002558, 24.85549872, 16.2544757 ],
        [30.38618926, 47.53069054, 31.0831202 ],
        [30.38618926, 47.53069054, 31.0831202 ],
        [11.42966752, 17.87851662, 11.69181586]]))

---

In [64]:
chisq_PUNTOS_Vi = pd.crosstab(df['Puntos Visitante'], df['resultado'])
chisq_PUNTOS_Vi

resultado,0,1,2
Puntos Visitante,,,
menos de 3,60,102,40
de 3 a 5,95,139,87
6 o más,63,100,96


In [65]:
scipy.stats.contingency.chi2_contingency(chisq_PUNTOS_Vi)

(17.684435221810404,
 0.0014222195585764852,
 4,
 array([[ 56.31202046,  88.08439898,  57.60358056],
        [ 89.4859335 , 139.97570332,  91.53836317],
        [ 72.20204604, 112.9398977 ,  73.85805627]]))

---

## Último partido
---

In [66]:
chisq_UPL = pd.crosstab(df['último partido como local'], df['resultado'])
chisq_UPL

resultado,0,1,2
último partido como local,,,
0,72,95,60
1,64,79,78
3,82,167,85


In [67]:
scipy.stats.contingency.chi2_contingency(chisq_UPL)
#p-value= 0.03
#critical value = 7.0

(14.05016322135952,
 0.007136664474913709,
 4,
 array([[ 63.28132992,  98.9859335 ,  64.73273657],
        [ 61.60869565,  96.36956522,  63.02173913],
        [ 93.10997442, 145.64450128,  95.2455243 ]]))

---

In [68]:
chisq_UP = pd.crosstab(df['último partido'], df['resultado'])
chisq_UP

resultado,0,1,2
último partido,,,
0,82,137,101
1,57,88,62
3,79,116,60


In [69]:
scipy.stats.contingency.chi2_contingency(chisq_UP)

(5.19704160874133,
 0.2676706610011784,
 4,
 array([[ 89.20716113, 139.53964194,  91.25319693],
        [ 57.70588235,  90.26470588,  59.02941176],
        [ 71.08695652, 111.19565217,  72.7173913 ]]))

---

In [70]:
chisq_UPV = pd.crosstab(df['último partido.1'], df['resultado'])
chisq_UPV

resultado,0,1,2
último partido.1,,,
0,72,109,66
1,70,93,64
3,76,139,93


In [71]:
scipy.stats.contingency.chi2_contingency(chisq_UPV)

(3.122879517711342,
 0.5374764936654439,
 4,
 array([[ 68.85677749, 107.70716113,  70.43606138],
        [ 63.28132992,  98.9859335 ,  64.73273657],
        [ 85.86189258, 134.30690537,  87.83120205]]))

---

# Goles

## Local
---

In [72]:
chisq_gfl = pd.crosstab(df['GF_local'], df['resultado'])
chisq_gfl

resultado,0,1,2
GF_local,,,
0-1.33,139,187,164
>1.33,79,154,59


In [73]:
scipy.stats.contingency.chi2_contingency(chisq_gfl)

(20.316225958467484,
 3.876034076809873e-05,
 2,
 array([[136.59846547, 213.67007673, 139.7314578 ],
        [ 81.40153453, 127.32992327,  83.2685422 ]]))

---

In [74]:
chisq_gcl = pd.crosstab(df['GC_local'], df['resultado'])
chisq_gcl

resultado,0,1,2
GC_local,,,
"(-0.001, 1.309]",135,258,154
"(1.309, 4.0]",83,83,69


In [75]:
scipy.stats.contingency.chi2_contingency(chisq_gcl)

(12.049392653146626,
 0.002418285825014783,
 2,
 array([[152.48849105, 238.52557545, 155.9859335 ],
        [ 65.51150895, 102.47442455,  67.0140665 ]]))

---

## Visitante

---

In [76]:
chisq_gfv = pd.crosstab(df['GF_visitante'], df['resultado'])
chisq_gfv

resultado,0,1,2
GF_visitante,,,
0-1.1,133,247,94
>1.1,85,94,129


In [77]:
scipy.stats.contingency.chi2_contingency(chisq_gfv)

(51.80679915291836,
 5.627253777898079e-12,
 2,
 array([[132.13810742, 206.69309463, 135.16879795],
        [ 85.86189258, 134.30690537,  87.83120205]]))

---

In [78]:
chisq_gcv = pd.crosstab(df['GC_visitante'], df['resultado'])
chisq_gcv

resultado,0,1,2
GC_visitante,,,
0-1.1,122,170,140
>1.4,96,171,83


In [79]:
scipy.stats.contingency.chi2_contingency(chisq_gcv)

(9.17578337978834,
 0.010174286327730537,
 2,
 array([[120.42966752, 188.37851662, 123.19181586],
        [ 97.57033248, 152.62148338,  99.80818414]]))

---

## Desempeño del equipo

In [80]:
chisq_mpsp_l = pd.crosstab(df['MPSP_l'], df['resultado'])
chisq_mpsp_l

resultado,0,1,2
MPSP_l,,,
"(0.999, 2.27]",94,109,110
"(2.27, 12.0]",124,232,113


In [81]:
scipy.stats.contingency.chi2_contingency(chisq_mpsp_l)

(18.13693467327196,
 0.00011524303213243223,
 2,
 array([[ 87.25575448, 136.48721228,  89.25703325],
        [130.74424552, 204.51278772, 133.74296675]]))

---

In [82]:
chisq_mpsg_l = pd.crosstab(df['MPSG_l'], df['resultado'])
chisq_mpsg_l

resultado,0,1,2
MPSG_l,,,
"(-0.001, 2.25]",67,190,63
"(2.25, 11.0]",151,151,160


In [83]:
scipy.stats.contingency.chi2_contingency(chisq_mpsg_l)

(55.0502329655245,
 1.1117158506302836e-12,
 2,
 array([[ 89.20716113, 139.53964194,  91.25319693],
        [128.79283887, 201.46035806, 131.74680307]]))

---

In [84]:
chisq_mpsp_v = pd.crosstab(df['MPSP_v'], df['resultado'])
chisq_mpsp_v

resultado,0,1,2
MPSP_v,,,
"(0.999, 2.233]",77,186,50
"(2.233, 12.0]",141,155,173


In [85]:
scipy.stats.contingency.chi2_contingency(chisq_mpsp_v)

(60.74750427585522,
 6.439424443461516e-14,
 2,
 array([[ 87.25575448, 136.48721228,  89.25703325],
        [130.74424552, 204.51278772, 133.74296675]]))

---

In [86]:
chisq_mpsg_v = pd.crosstab(df['MPSG_v'], df['resultado'])
chisq_mpsg_v

resultado,0,1,2
MPSG_v,,,
"(-0.001, 2.25]",87,94,134
"(2.25, 11.0]",131,247,89


In [87]:
scipy.stats.contingency.chi2_contingency(chisq_mpsg_v)

(59.30540696268806,
 1.3243247773918571e-13,
 2,
 array([[ 87.81329923, 137.35933504,  89.82736573],
        [130.18670077, 203.64066496, 133.17263427]]))

---